In [1]:
import numpy as np
import pandas as pd
import os
import shutil
import random
import torch
import torchvision
import matplotlib.pyplot as plt
import time
import copy
import PIL

from torchvision import transforms, models
from tqdm import tqdm_notebook as tqdm

data_root = './data/initial'
data_generated = './data/generated'
train_dir = './data/train'
val_dir = './data/val'
test_dir = './data/initial/test'
model_dir = './best_model'

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [2]:
train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = torchvision.datasets.ImageFolder(os.path.join('./cheating_data/generated', 'train'), train_transforms)
val_dataset = torchvision.datasets.ImageFolder('./cheating_data/val', val_transforms)

In [3]:
def validate_model(model, loss, dataloader, device):
    was_training = model.training
    model.eval()
    running_loss = 0
    running_acc = 0
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        with torch.set_grad_enabled(False):
            preds = model(inputs)
            loss_value = loss(preds, labels)
            preds_class = preds.argmax(dim=1)
            running_loss += loss_value.item()
            running_acc += (preds_class == labels.data).float().mean()
    epoch_loss = running_loss / len(dataloader)
    epoch_acc = running_acc / len(dataloader)
    if was_training:
        model.train()
    return epoch_loss, epoch_acc

def save_model(model, val_acc, path):
    shutil.rmtree(path, ignore_errors=True)
    os.makedirs(path)
    model_name = 'model_{:.3f}.torch'.format(val_acc)
    torch.save({'state_dict': model.state_dict()}, os.path.join(path, model_name))

def load_model(model_cls, path, device):
    model = model_cls()
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['state_dict'])
    model.eval()
    return model.to(device)

In [25]:
def train_model(model, loss, optimizer, scheduler, num_epochs, eval_every_nth_batch, stop_after_steps=None):
    best_val_acc = 0
    for epoch in range(num_epochs):
        print('Epoch {}/{}:'.format(epoch, num_epochs - 1), flush=True)

        dataloader = train_dataloader
        scheduler.step()
        model.train()

        running_loss = 0
        running_acc = 0
        batch_id = 0
        n_samples_passed = 0
        for inputs, labels in tqdm(dataloader):
            if stop_after_steps is not None and stop_after_steps <= 0:
                model.eval()
                return model
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            preds = model(inputs)
            loss_value = loss(preds, labels)
            preds_class = preds.argmax(dim=1)

            loss_value.backward()
            optimizer.step()

            running_loss += loss_value.item()
            running_acc += (preds_class == labels.data).float().mean()
            batch_id += 1
            if batch_id % eval_every_nth_batch == 0:
                train_loss = running_loss / eval_every_nth_batch
                train_acc = running_acc / eval_every_nth_batch
                
                val_loss, val_acc = validate_model(model, loss, val_dataloader, device)
                is_best = val_acc > best_val_acc
                if is_best:
                    save_model(model, val_acc, model_dir)
                    best_val_acc = val_acc
                    val_loss_msg = 'Valid loss: {:.4f} Valid acc: {:.4f} ***'.format(val_loss, val_acc)
                    val_loss_msg += ' CURRENTLY THE BEST!'
                    print('Train loss: {:.4f} Train acc: {:.4f}'.format(train_loss, train_acc), flush=True)
                    print(val_loss_msg, flush=True)
                
                running_loss = 0
                running_acc = 0
            stop_after_steps = stop_after_steps if stop_after_steps is None else stop_after_steps - 1

    return model

In [38]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.__head = models.resnet50(pretrained=True)
        for param in self.__head.parameters():
            param.requires_grad = False
        
        self.__bottom = torch.nn.Sequential()
        self.__bottom.add_module("linear_1", torch.nn.Linear(self.__head.fc.in_features, 256))
        self.__bottom.add_module("linear_1_dropout", torch.nn.Dropout(p=0.3))
        self.__bottom.add_module("relu_1", torch.nn.ReLU(inplace=True))
        self.__bottom.add_module("linear_2", torch.nn.Linear(256, 2))
        
        self.__head.fc = self.__bottom
        
    def prepare_to_fine_tune(self):
        for param in model.__head.layer4[2].parameters():
            param.requires_grad = True
                
    def stop_fine_tune(self):
        for param in model.__head.layer4[2].parameters():
            param.requires_grad = False
    
    def forward(self, x):
        return self.__head.forward(x)

model = Model()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [43]:
model.prepare_to_fine_tune()
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

batch_size = 64
eval_every_nth_batch = 1
n_workers = 4
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=n_workers)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=n_workers)

train_model(model, loss, optimizer, scheduler, num_epochs=1, eval_every_nth_batch=eval_every_nth_batch, stop_after_steps=100);

Epoch 0/0:


A Jupyter Widget

Train loss: 0.0274 Train acc: 1.0000
Valid loss: 0.1989 Valid acc: 0.9225 *** CURRENTLY THE BEST!
Train loss: 0.0839 Train acc: 0.9531
Valid loss: 0.1992 Valid acc: 0.9303 *** CURRENTLY THE BEST!
Train loss: 0.0418 Train acc: 0.9688
Valid loss: 0.1992 Valid acc: 0.9321 *** CURRENTLY THE BEST!
Train loss: 0.0314 Train acc: 0.9844
Valid loss: 0.1991 Valid acc: 0.9399 *** CURRENTLY THE BEST!
Train loss: 0.0829 Train acc: 0.9688
Valid loss: 0.1965 Valid acc: 0.9417 *** CURRENTLY THE BEST!


In [45]:
model = load_model(Model, './best_model/model_0.942.torch', device)

In [46]:
model.stop_fine_tune()
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, amsgrad=True, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [47]:
batch_size = 64
eval_every_nth_batch = 1
n_workers = 4
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=n_workers)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=n_workers)

train_model(model, loss, optimizer, scheduler, num_epochs=25, eval_every_nth_batch=eval_every_nth_batch);

Epoch 0/24:


A Jupyter Widget

Train loss: 0.0263 Train acc: 1.0000
Valid loss: 0.4579 Valid acc: 0.8960 *** CURRENTLY THE BEST!
Train loss: 0.1612 Train acc: 0.9531
Valid loss: 0.2072 Valid acc: 0.9225 *** CURRENTLY THE BEST!
Train loss: 0.2784 Train acc: 0.8906
Valid loss: 0.2288 Valid acc: 0.9285 *** CURRENTLY THE BEST!
Train loss: 0.0721 Train acc: 0.9844
Valid loss: 0.2004 Valid acc: 0.9303 *** CURRENTLY THE BEST!
Train loss: 0.1768 Train acc: 0.9219
Valid loss: 0.1999 Valid acc: 0.9381 *** CURRENTLY THE BEST!
Train loss: 0.1923 Train acc: 0.9219
Valid loss: 0.1972 Valid acc: 0.9477 *** CURRENTLY THE BEST!
Epoch 1/24:


A Jupyter Widget

Epoch 2/24:


...

Epoch 14/24:


A Jupyter Widget

Epoch 15/24:


A Jupyter Widget

KeyboardInterrupt: 

In [30]:
model = load_model(Model, './best_model/model_0.965.torch', device)

In [48]:
model = load_model(Model, './resnet50_0.973_0.965.torch', device)

In [56]:
test_transforms = transforms.Compose([
    transforms.ColorJitter(brightness=0.3, hue=0.25),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [57]:
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = os.path.basename(self.imgs[index][0])
        path = path.split()[0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path
    
test_dataset = ImageFolderWithPaths('../data/initial/test', test_transforms)
#test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

In [52]:
n_runs_per_sample = 10
model.eval()

test_predictions = None
for _ in range(n_runs_per_sample):
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)
    temp_predictions = []
    test_img_paths = []
    for inputs, labels, paths in tqdm(test_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        with torch.set_grad_enabled(False):
            preds = model(inputs)
        temp_predictions.append(torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy())
        test_img_paths.extend(paths)

    mean_predictions = np.concatenate(temp_predictions) / n_runs_per_sample
    test_predictions = mean_predictions if test_predictions is None else test_predictions + mean_predictions

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [34]:
model.eval()

test_predictions = []
test_img_paths = []
for inputs, labels, paths in tqdm(test_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.set_grad_enabled(False):
        preds = model(inputs)
    test_predictions.append(
        torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy())
    test_img_paths.extend(paths)
    
test_predictions = np.concatenate(test_predictions)

A Jupyter Widget

In [53]:
submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 'label': test_predictions})

In [54]:
submission_df['label'] = submission_df['label'].map(lambda pred: 'dirty' if pred > 0.5 else 'cleaned')
submission_df['id'] = submission_df['id'].str.replace('test/unknown/', '')
submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
submission_df.set_index('id', inplace=True)
submission_df.head(n=6)

,label
id,
0000,dirty
0001,dirty
0002,dirty
0003,dirty
0004,dirty
0005,dirty


In [55]:
submission_df.to_csv('submission.csv')

# Data Preparation

In [12]:
class_names = ['cleaned', 'dirty']

for dir_name in [train_dir, val_dir]:
    for class_name in class_names:
        os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)

for class_name in class_names:
    source_dir = os.path.join(data_root, 'train', class_name)
    for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
        if i % 6 != 0:
            dest_dir = os.path.join(train_dir, class_name) 
        else:
            dest_dir = os.path.join(val_dir, class_name)
        shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))

100%|███████████████████████████████████████| 20/20 [00:00<00:00, 172.42it/s]


Let's generate some samples

In [2]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

def prepare_dirs(root_path):
    shutil.rmtree(root_path, ignore_errors=True)
    os.makedirs(os.path.join(root_path, 'dirty'))
    os.makedirs(os.path.join(root_path, 'clean'))

def make_dataset(directory):
    ts = transforms.Compose([
        transforms.ColorJitter(brightness=0.3, hue=0.25),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(180.0, resample=PIL.Image.BICUBIC),
        transforms.Resize((224, 224)),
        transforms.RandomAffine(degrees=0, translate=(0.15, 0.15))
    ])
    return torchvision.datasets.ImageFolder(directory, ts)

def generate_images(n_amount, dataset_factory, dataset_type, root_path):
    path = os.path.join(root_path, dataset_type)
    prepare_dirs(path)
    name_pattern = '{{:0{}d}}.png'.format(len(str(n_amount)))
    class_path = {0: 'clean', 1: 'dirty'}
    image_id = 0
    while True:
        dataset = dataset_factory()
        data_iter = iter(dataset)
        for image, class_id in data_iter:
            if image_id >= n_amount:
                return
            image_name = name_pattern.format(image_id)
            image, class_id = next(data_iter)
            image.save(os.path.join(path, class_path[class_id], image_name), 'PNG')
            image_id += 1

In [5]:
n_gen_images_train = 8192
n_gen_images_val = 4096

def make_train_dataset():
    return make_dataset('./cheating_data/train/')

def make_val_dataset():
    return make_dataset(val_dir)

data_generated = './cheating_data/generated'
generate_images(n_gen_images_train, make_train_dataset, 'train', data_generated)
#generate_images(n_gen_images_val, make_val_dataset, 'val', data_generated)